In [1]:
import json
import pandas as pd
import numpy as np

import re

from sqlalchemy import create_engine
import psycopg2

# from config import db_password

import time

In [2]:
#  Add the clean movie function that takes in the argument, "movie".
def clean_movie(movie):
    movie = dict(movie) #create a non-destructive copy
    alt_titles = {}
    # combine alternate titles into one list
    for key in ['Also known as','Arabic','Cantonese','Chinese','French',
                'Hangul','Hebrew','Hepburn','Japanese','Literally',
                'Mandarin','McCune-Reischauer','Original title','Polish',
                'Revised Romanization','Romanized','Russian',
                'Simplified','Traditional','Yiddish']:
        if key in movie:
            alt_titles[key] = movie[key]
            movie.pop(key)
    if len(alt_titles) > 0:
        movie['alt_titles'] = alt_titles

    # merge column names
    def change_column_name(old_name, new_name):
        if old_name in movie:
            movie[new_name] = movie.pop(old_name)
    change_column_name('Adaptation by', 'Writer(s)')
    change_column_name('Country of origin', 'Country')
    change_column_name('Directed by', 'Director')
    change_column_name('Distributed by', 'Distributor')
    change_column_name('Edited by', 'Editor(s)')
    change_column_name('Length', 'Running time')
    change_column_name('Original release', 'Release date')
    change_column_name('Music by', 'Composer(s)')
    change_column_name('Produced by', 'Producer(s)')
    change_column_name('Producer', 'Producer(s)')
    change_column_name('Productioncompanies ', 'Production company(s)')
    change_column_name('Productioncompany ', 'Production company(s)')
    change_column_name('Released', 'Release Date')
    change_column_name('Release Date', 'Release date')
    change_column_name('Screen story by', 'Writer(s)')
    change_column_name('Screenplay by', 'Writer(s)')
    change_column_name('Story by', 'Writer(s)')
    change_column_name('Theme music composer', 'Composer(s)')
    change_column_name('Written by', 'Writer(s)')
    
    
    

    return movie

In [3]:
# 1 Add the function that takes in three arguments;
# Wikipedia data, Kaggle metadata, and MovieLens rating data (from Kaggle)

def extract_transform_load():
    # Read in the kaggle metadata and MovieLens ratings CSV files as Pandas DataFrames.
    global kaggleMeta
    kaggleMeta=pd.read_csv(kaggleFile, low_memory=False)
    global ratingsDf
    ratingsDf=pd.read_csv(movieLens)


    # Open and read the Wikipedia data JSON file.
    with open (wikiMovies) as file:
        wikiMoviesReadIn=json.load(file) 
    
    # Write a list comprehension to filter out TV shows.
    wikiMoviesForCleaning=[movie for movie in wikiMoviesReadIn
                        if 'No. of episodes' not in movie]

    # Write a list comprehension to iterate through the cleaned wiki movies list
    # and call the clean_movie function on each movie.
    wikiMoviesClean= [clean_movie(movie) for movie in wikiMoviesForCleaning]

    # Read in the cleaned movies list from Step 4 as a DataFrame.
    global wikiMoviesDf
    wikiMoviesDf=pd.DataFrame(wikiMoviesClean)

    # Write a try-except block to catch errors while extracting the IMDb ID using a regular expression string and
    #  dropping any imdb_id duplicates. If there is an error, capture and print the exception.
    try: 
        wikiMoviesDf['imdb_id']=wikiMoviesDf['imdb_link'].str.extract(r'(tt\d{7})')
        wikiMoviesDf.drop_duplicates(subset='imdb_id', inplace=True)
        
    except Exception as e: 
        print (e)

    #  Write a list comprehension to keep the columns that don't have null values from the wiki_movies_df DataFrame.
    wikiColumnsDropNA=[column for column in wikiMoviesDf.columns if wikiMoviesDf[column].isnull().sum()<len(wikiMoviesDf)*0.9]
    wikiMoviesDf=wikiMoviesDf[wikiColumnsDropNA]

    # Create a variable that will hold the non-null values from the “Box office” column.
    boxOffice=wikiMoviesDf['Box office'].dropna()
    
    # Convert the box office data created in Step 8 to string values using the lambda and join functions.
    boxOffice=boxOffice.apply(lambda x:' '.join(x) if type(x)==list else x)

    # Write a regular expression to match the six elements of "form_one" of the box office data.
    form_one=r'\$\s*\d+\.?\d*\s*[mb]illi?on'
    # Write a regular expression to match the three elements of "form_two" of the box office data.
    form_two =r'\$\s*\d{1,3}(?:[,\.]\d{3})+(?!\s[mb]illion)'

    # Add the parse_dollars function.
    def parse_dollars(s):
    # if s is not a string, return NaN
        if type(s) != str:
            return np.nan

    # if input is of the form $###.# million
        if re.match(r'\$\s*\d+\.?\d*\s*milli?on', s, flags=re.IGNORECASE):

        # remove dollar sign and " million"
            s = re.sub('\$|\s|[a-zA-Z]','', s)

        # convert to float and multiply by a million
            value = float(s) * 10**6

        # return value
            return value

    # if input is of the form $###.# billion
        elif re.match(r'\$\s*\d+\.?\d*\s*billi?on', s, flags=re.IGNORECASE):

        # remove dollar sign and " billion"
            s = re.sub('\$|\s|[a-zA-Z]','', s)

        # convert to float and multiply by a billion
            value = float(s) * 10**9

        # return value
            return value

    # if input is of the form $###,###,###
        elif re.match(r'\$\s*\d{1,3}(?:[,\.]\d{3})+(?!\s[mb]illion)', s, flags=re.IGNORECASE):

        # remove dollar sign and commas
            s = re.sub('\$|,','', s)

        # convert to float
            value = float(s)

        # return value
            return value

    # otherwise, return NaN
        else:
            return np.nan
        
    # Clean the box office column in the wiki_movies_df DataFrame.
    wikiMoviesDf['boxOffice'] = boxOffice.str.extract(f'({form_one}|{form_two})', flags=re.IGNORECASE)[0].apply(parse_dollars)
    
    # Clean the budget column in the wiki_movies_df DataFrame.
    #regex to reformat Budget column into new column budget which resembles form_one or form_two variables 
    budget=wikiMoviesDf['Budget'].dropna()\
    .apply(lambda x: ' '.join(x) if type(x) == list else x)\
    .str.replace(r'\$.*[-—–](?![a-z])', '$', regex=True)\
    .str.replace(r'\[\d+\]\s*', '')
    #passing budget through parse_dollars function
    wikiMoviesDf['budget'] = budget.str.extract(f'({form_one}|{form_two})', flags=re.IGNORECASE)[0].apply(parse_dollars)

    # Clean the release date column in the wiki_movies_df DataFrame.
    date_form_one = r'(?:January|February|March|April|May|June|July|August|September|October|November|December)\s[0123]?\d,\s\d{4}'
    date_form_two = r'\d{4}.[01]\d.[0123]\d'
    date_form_three = r'(?:January|February|March|April|May|June|July|August|September|October|November|December)\s\d{4}'
    date_form_four = r'\d{4}'
    wikiMoviesDf['releaseDate']=wikiMoviesDf['Release date'].dropna()\
    .apply(lambda x: ' '.join(x)if type(x)==list else x)\
    .str.extract(f'({date_form_one}|{date_form_two}|{date_form_three}|{date_form_four})', flags=re.IGNORECASE)
    #and converting to datetime
    wikiMoviesDf['releaseDate']=pd.to_datetime(wikiMoviesDf['releaseDate'])
    # Clean the running time column in the wiki_movies_df DataFrame.
    wikiMoviesDf['runtime']=wikiMoviesDf['Running time'].dropna()\
    .str.extract(r'(\d+)\s*ho?u?r?s?\s*(\d*)|(\d+)\s*m')\
    .apply(lambda col: pd.to_numeric(col, errors='coerce')).fillna(0)\
    .apply(lambda row: row[0]*60 + row[1] if row[2] == 0 else row[2], axis=1)
     
    # 2. Clean the Kaggle metadata.
    kaggleMeta=kaggleMeta[kaggleMeta['adult']=='False'].drop('adult',axis='columns')
    kaggleMeta['video']=kaggleMeta['video']=='True'
    kaggleMeta['budget']=kaggleMeta['budget'].astype(int)
    kaggleMeta['id']=pd.to_numeric(kaggleMeta['id'], errors='raise')
    kaggleMeta['popularity']=pd.to_numeric(kaggleMeta['popularity'], errors='raise')
    kaggleMeta['release_date']=pd.to_datetime(kaggleMeta['release_date'])
    # 3. Merged the two DataFrames into the movies DataFrame.
    global moviesDf
    moviesDf=pd.merge(wikiMoviesDf,kaggleMeta,on='imdb_id',suffixes=['_wiki','_kaggle'])

    # 4. Drop unnecessary columns from the merged DataFrame.
    moviesDf.drop(columns=['title_wiki','releaseDate','Language','Production company(s)','Box office',\
                  'Budget','Release date','Running time'], inplace= True)

    # 5. Add in the function to fill in the missing Kaggle data.
    def fill_missing_kaggle_data(df, kaggle_column, wiki_column):
        df[kaggle_column] = df.apply(
            lambda row: row[wiki_column] if row[kaggle_column] == 0 else row[kaggle_column]
            , axis=1)
        df.drop(columns=wiki_column, inplace= True)

    # 6. Call the function in Step 5 with the DataFrame and columns as the arguments.
    fill_missing_kaggle_data(moviesDf,'runtime_kaggle','runtime_wiki')
    fill_missing_kaggle_data(moviesDf,'budget_kaggle','budget_wiki')
    fill_missing_kaggle_data(moviesDf,'revenue','boxOffice')
    # 7. Filter the movies DataFrame for specific columns.
    moviesDf=moviesDf.filter(items=['id','title_kaggle','tagline','url','imdb_link','runtime','budget_kaggle','revenue','release_date',\
                           'popularity','genres','overview','Based on','Starring','Country',\
                           'Distributor','Director','Editor(s)','Writer(s)','Composer(s)'])
    # 8. Rename the columns in the movies DataFrame.
    moviesDf=moviesDf.rename({'id':'kaggle_id',
                             'title_kaggle':'title',
                             'url':'wikipedia_url',
                             'budget_kaggle':'budget',
                             'Based on':'based_on',
                             'Starring':'starring',
                             'Country':'country',
                             'Distributor':'distributor',
                             'Director':'director',
                             'Editor(s)':'editors',
                             'Writer(s)':'writers',
                             'Composer(s)':'composers'},inplace=True)

    # 9. Transform and merge the ratings DataFrame.
    ratingCounts = ratingsDf.groupby(['movieId','rating'], as_index=False).count() \
                .rename({'userId':'count'}, axis=1) \
                .pivot(index='movieId',columns='rating', values='count')
    ratingCounts.columns = ['rating_' + str(col) for col in ratingCounts.columns]
    moviesWithRatingsDf = pd.merge(moviesDf,ratingCounts, left_on='id', right_index=True, how='left')
    moviesWithRatingsDf[ratingCounts.columns] = moviesWithRatingsDf[ratingCounts.columns].fillna(0)
    return wikiMoviesDf, kaggleMeta, ratingsDf


In [4]:
# 10. Create the path to your file directory and variables for the three files.
file_dir ='C:/Users/chris/ClassTemp/Movies-ETL' 
# Wikipedia data
wikiMovies = f'{file_dir}/wikipedia-movies.json'
# The Kaggle metadata
kaggleFile = f'{file_dir}/movies_metadata.csv'
# The MovieLens rating data.
movieLens = f'{file_dir}/ratings.csv'

In [5]:
# 11. Set the three variables equal to the function created in D1.
wikiMovies,kaggleFile,movieLens = extract_transform_load()

C:\Users\chris\.conda\envs\PythonData\lib\site-packages\ipykernel_launcher.py:106: FutureWarning: The default value of regex will change from True to False in a future version.


TypeError: Can only merge Series or DataFrame objects, a <class 'NoneType'> was passed

In [ ]:
# 12. Set the DataFrames from the return statement equal to the file names in Step 11. 
wikiMoviesDf=wikiMovies
kaggleMeta = kaggleFile
ratingsDf = movieLens

In [ ]:
# 13. Check the wiki_movies_df DataFrame. 
wikiMoviesDf.head()

In [ ]:
# 14. Check the movies_with_ratings_df DataFrame.
ratingsDf.head()

In [ ]:
# 15. Check the movies_df DataFrame. 
moviesDf.head()

In [ ]:
kaggleMeta.columns.tolist

In [ ]:
moviesDf[['Distributor','production_companies']].head(100)

In [ ]:
moviesDf['budget_kaggle']

In [ ]:
moviesWithRatingsDf